# TypeCtx SRC
We will first take a script and add the Cassette logic to get the information from the stack trace.

In [1]:
using Cassette;
using DifferentialEquations: ODEProblem, solve

In [2]:
function main()
    
    # define our ode
    function sir_ode(du, u, p, t)  
        #Infected per-Capita Rate
        β = p[1]
        #Recover per-capita rate
        γ = p[2]
        #Susceptible Individuals
        S = u[1]
        #Infected by Infected Individuals
        I = u[2]

        du[1] = -β * S * I
        du[2] = β * S * I - γ * I
        du[3] = γ * I
    end

    #Pram = (Infected Per Capita Rate, Recover Per Capita Rate)
    pram = [0.1,0.05]
    #Initial Prams = (Susceptible Individuals, Infected by Infected Individuals)
    init = [0.99,0.01,0.0]
    tspan = (0.0,200.0)
    
    # create a var to our problem
    sir_prob = ODEProblem(sir_ode, init, tspan, pram)
    solution = solve(sir_prob)
    
end

main (generic function with 1 method)

In [2]:
mutable struct FCollector{I,F,C}
    depth::I
    frame::F
    data::Vector{C}
end

In [3]:
function FCollector(d::Int, f)
    FCollector(d, f, FCollector[])
end

FCollector

In [4]:
""" 
Frame(func, args, ret, subtrace)

a structure to hold metadata for recursive type information
"""
mutable struct Frame{F,T,U}
    func::F
    args::T
    ret::U
end

Frame

In [5]:
# user 
Cassette.@context TypeCtx;

In [15]:
# user
maxdepth =3
extractor = FCollector(maxdepth, Frame(nothing, (), nothing,))

FCollector{Int64,Frame{Nothing,Tuple{},Nothing},FCollector}(3, Frame{Nothing,Tuple{},Nothing}(nothing, (), nothing), FCollector[])

In [16]:
ctx = TypeCtx(metadata = extractor);

In [8]:
# add boilerplate for functionality
function Cassette.overdub(ctx::TypeCtx, f, args...)
    c = FCollector(ctx.metadata.depth-1, Frame(f, args, Any))
    push!(ctx.metadata.data, c)
    if c.depth > 0 && Cassette.canrecurse(ctx, f, args...)
        newctx = Cassette.similarcontext(ctx, metadata = c)
        z = Cassette.recurse(newctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    else
        z = Cassette.fallback(ctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    end
end

In [9]:
Cassette.canrecurse(ctx::TypeCtx,::typeof(ODEProblem), args...) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Base.vect), args...) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(FCollector)) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Frame)) = false

In [17]:
Cassette.overdub(ctx,main)

retcode: Success
Interpolation: 1st order linear
t: 2001-element Array{Float64,1}:
   0.0
   0.1
   0.2
   0.3
   0.4
   0.5
   0.6
   0.7
   0.8
   0.9
   1.0
   1.1
   1.2
   ⋮  
 198.9
 199.0
 199.1
 199.2
 199.3
 199.4
 199.5
 199.6
 199.7
 199.8
 199.9
 200.0
u: 2001-element Array{Array{Float64,1},1}:
 [0.99, 0.01, 0.0]                 
 [0.989901, 0.0100491, 5.01227e-5] 
 [0.989801, 0.0100985, 0.000100492]
 [0.989701, 0.010148, 0.000151108] 
 [0.9896, 0.0101979, 0.000201972]  
 [0.989499, 0.0102479, 0.000253087]
 [0.989397, 0.0102982, 0.000304452]
 [0.989295, 0.0103487, 0.000356069]
 [0.989193, 0.0103995, 0.000407939]
 [0.989089, 0.0104504, 0.000460064]
 [0.988986, 0.0105017, 0.000512444]
 [0.988882, 0.0105531, 0.000565081]
 [0.988777, 0.0106049, 0.000617976]
 ⋮                                 
 [0.21018, 0.0149422, 0.774878]    
 [0.210148, 0.014899, 0.774953]    
 [0.210117, 0.0148558, 0.775027]   
 [0.210086, 0.0148128, 0.775101]   
 [0.210055, 0.01477, 0.775175]     
 [0.2100

In [19]:
Cassette.overdub(ctx,main);

In [7]:
using LightGraphs;
using MetaGraphs;

In [13]:
g = MetaDiGraph()
function build_graph(collector::FCollector)
    add_vertex!(g,:name,collector.frame.args)
    add_vertex!(g,:name,collector.frame.ret)
    add_edge!(g,nv(g)-1,nv(g),:name,collector.frame.func)
    for frame in collector.data
        build_graph(frame)
    end
    return g
end

build_graph (generic function with 1 method)

In [14]:
build_graph(extractor)

{8, 4} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [ ]:
using Cassette
using SemanticModels.Parsers
Cassette.@context TypeCtx
            
"""   TypeCtx

creates a MetaDiGraph tracking the types of args and ret values throughout a script

"""
TypeCtx
            
            

"""   FCollector(depth::Int,frame::function,data::FCollector)

struct to collect all the "frames" called throughout a script
        
"""
mutable struct FCollector{I,F,C}
    depth::I
    frame::F
    data::Vector{C}
end
            


"""   FCollector(depth::Int,frame::Frame)

this is an initialization funtion for the FCollector

"""
function FCollector(d::Int, f)
    FCollector(d, f, FCollector[])
end

""" Frame(func, args, ret, subtrace)

a structure to hold metadata for recursive type information for each function call
Every frame can be thought of as a single stack frame when a function is called
            
"""
mutable struct Frame{F,T,U}
    func::F
    args::T
    ret::U
end
            
function Cassette.overdub(ctx::TypeCtx, f, args...) # add boilerplate for functionality
    c = FCollector(ctx.metadata.depth-1, Frame(f, args, Any))
    push!(ctx.metadata.data, c)
    if c.depth > 0 && Cassette.canrecurse(ctx, f, args...)
        newctx = Cassette.similarcontext(ctx, metadata = c)
        z = Cassette.recurse(newctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    else
        z = Cassette.fallback(ctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    end
end

Cassette.canrecurse(ctx::TypeCtx,::typeof(Base.vect), args...) = false # limit the stacktrace in terms of which to recurse on
Cassette.canrecurse(ctx::TypeCtx,::typeof(FCollector)) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Frame)) = false
     
"""    buildgraph

internal function used in the typegraphfrompath
takes the collector object and returns a metagraph
            
"""
function buildgraph(g,collector)
    try
        add_vertex!(g,:name,collector.frame.args)
    catch
        nothing
    end
    try
        add_vertex!(g,:name,collector.frame.ret)
    catch
        nothing
    end
    try
        add_edge!(g,g[collector.frame.args,:name],g[collector.frame.ret,:name],:name,collector.frame.func)
    catch
        nothing
    end
    for frame in collector.data
        buildgraph(g,frame)
    end
    return g
end

"""    typegraph(path::AbstractString,maxdepth::Int)
            
This is a function that takes in an array of script and produces a MetaDiGraph descibing the system.
takes in optional parameter of recursion depth on the stacktrace defaulted to 3

"""
function typegraph(path::Module,maxdepth::Int=3)
    
    extractor = FCollector(maxdepth, Frame(nothing, (), nothing,)) # init the collector object     
    ctx = TypeCtx(metadata = extractor);     # init the context we want             
    Cassette.overdub(ctx,m.main);    # run the script internally and build the extractor data structure
    g = MetaDiGraph()    # crete a graph where we will init our tree
    set_indexing_prop!(g,:name)    # we want to set this metagraph to be able to index by the names
    return buildgraph(g,extractor)    # pass the collector ds to make the acutal metagraph
    
end

┌ Info: Recompiling stale cache file /home/infvie/.julia/compiled/v1.0/SemanticModels/0ZHVR.ji for SemanticModels [88974b6f-63f0-5f7a-882c-b779d2960b8e]
└ @ Base loading.jl:1190


# SRC Evaluation

In [1]:
using SemanticModels.Dubstep

┌ Info: Precompiling SemanticModels [88974b6f-63f0-5f7a-882c-b779d2960b8e]
└ @ Base loading.jl:1192


loaded


In [4]:
files = ["files/$file" for file in readdir("./files")]

16-element Array{String,1}:
 "files/.ipynb_checkpoints"                     
 "files/ContinuousTimeSIR.jl"                   
 "files/DiscreteStochErlangEpidModel.jl"        
 "files/DiscreteTimeSIR.jl"                     
 "files/ExactRecursiveExpressions-checkpoint.jl"
 "files/ExactRecursiveExpressions.jl"           
 "files/MacroParasiteModel.jl"                  
 "files/NHosts1Vector.jl"                       
 "files/NHostsMVectors.jl"                      
 "files/OneHostSEIR1Vect.jl"                    
 "files/SEIRmodel.jl"                           
 "files/SIRModel.jl"                            
 "files/SIRSDynamicsLargePopulationsJulia.jl"   
 "files/SISModel.jl"                            
 "files/ScalingModel.jl"                        
 "files/SemiParamSIRJulia.jl"                   

In [2]:
typegraph("sir.jl")

{2, 1} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [17]:
typegraph(files[3])

{6, 3} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [18]:
typegraph(files[4])

{6, 3} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

# Flat linear tests

In [1]:
using Cassette
using LightGraphs;
using MetaGraphs;
function parsefile(path, modprefix="Modeling")
    s = read(path, String)
    try
        expr = Meta.parse(s)
        return expr
    catch
        s = "module $modprefix\n$s \nend"
        expr = Meta.parse(s)
        return expr
    end
end

Cassette.@context TypeCtx
            
"""   TypeCtx

creates a MetaDiGraph tracking the types of args and ret values throughout a script

"""
TypeCtx
            
            

"""   FCollector(depth::Int,frame::function,data::FCollector)

struct to collect all the "frames" called throughout a script
        
"""
mutable struct FCollector{I,F,C}
    depth::I
    frame::F
    data::Vector{C}
end
            


"""   FCollector(depth::Int,frame::Frame)

this is an initialization funtion for the FCollector

"""
function FCollector(d::Int, f)
    FCollector(d, f, FCollector[])
end

""" Frame(func, args, ret, subtrace)

a structure to hold metadata for recursive type information for each function call
Every frame can be thought of as a single stack frame when a function is called
            
"""
mutable struct Frame{F,T,U}
    func::F
    args::T
    ret::U
end
            
function Cassette.overdub(ctx::TypeCtx, f, args...) # add boilerplate for functionality
    c = FCollector(ctx.metadata.depth-1, Frame(f, args, Any))
    push!(ctx.metadata.data, c)
    if c.depth > 0 && Cassette.canrecurse(ctx, f, args...)
        newctx = Cassette.similarcontext(ctx, metadata = c)
        z = Cassette.recurse(newctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    else
        z = Cassette.fallback(ctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    end
end

Cassette.canrecurse(ctx::TypeCtx,::typeof(Base.vect), args...) = false # limit the stacktrace in terms of which to recurse on
Cassette.canrecurse(ctx::TypeCtx,::typeof(FCollector)) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Frame)) = false
     
"""    buildgraph

internal function used in the typegraphfrompath
takes the collector object and returns a metagraph
            
"""
function buildgraph(g,collector)
    try
        add_vertex!(g,:name,collector.frame.args)
    catch
        nothing
    end
    try
        add_vertex!(g,:name,collector.frame.ret)
    catch
        nothing
    end
    try
        add_edge!(g,g[collector.frame.args,:name],g[collector.frame.ret,:name],:name,collector.frame.func)
    catch
        nothing
    end
    for frame in collector.data
        buildgraph(g,frame)
    end
    return g
end

typegraph

In [2]:
path = "sir.jl"

"sir.jl"

In [3]:
s = read(path, String)
ast = Meta.parse(s)
m = eval(ast)
Cassette.@context TypeCtx;
extractor = FCollector(3, Frame(nothing, (), nothing,)) # init the collector object     
ctx = TypeCtx(metadata = extractor);     # init the context we want         
ast = parsefile(path)    # this makes the path we are evaling into a mod if it isnt already
m = eval(ast)  # compile the funcs - requires main function in script
Cassette.overdub(ctx,m.main);    # run the script internally and build the extractor data structure
g = MetaDiGraph()    # crete a graph where we will init our tree
set_indexing_prop!(g,:name)    # we want to set this metagraph to be able to index by the names
buildgraph(g,extractor)    # pass the collector ds to make the acutal metagraph

{104, 31} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

# Memoize

In [1]:
using Cassette;
using LightGraphs;
using MetaGraphs;

In [15]:

#---------------------------------------------------------------------------
            
Cassette.@context TypeCtx
            
"""   TypeCtx

creates a MetaDiGraph tracking the types of args and ret values throughout a script

"""
TypeCtx
            
            

"""   FCollector(depth::Int,frame::function,data::FCollector)

struct to collect all the "frames" called throughout a script
        
"""
mutable struct FCollector{I,F,C}
    depth::I
    frame::F
    data::Vector{C}
end
            


"""   FCollector(depth::Int,frame::Frame)

this is an initialization funtion for the FCollector

"""
function FCollector(d::Int, f)
    FCollector(d, f, FCollector[])
end

"""    Frame(func, args, ret)

a structure to hold metadata for recursive type information for each function call
Every frame can be thought of as a single stack frame when a function is called
            
"""
mutable struct Frame{F,T,U}
    func::F
    args::T
    ret::Vector{U}
end
    
"""    memo_dict((func,typeof.(args))=>ret)   

A dictionary that we use to memoize the edge list as we go to save space in the overall collection.

"""
memo_dict = Dict()
            
function Cassette.overdub(ctx::TypeCtx, f, args...) # add boilerplate for functionality
    c = FCollector(ctx.metadata.depth-1, Frame(f, typeof.(args), []) )
    if !haskey(memo_dict,(f,typeof.(args)))
        push!(ctx.metadata.data, c)
    end
    if c.depth > 0 && Cassette.canrecurse(ctx, f, args...)
        newctx = Cassette.similarcontext(ctx, metadata = c)
        z = Cassette.recurse(newctx, f, args...)
        push!(c.frame.ret, typeof(z))
        return z
    else
        z = Cassette.fallback(ctx, f, args...)
        push!(c.frame.ret, typeof(z))
        return z
    end
end

Cassette.canrecurse(ctx::Type{TypeCtx},::typeof(Base.vect), args...) = false # limit the stacktrace in terms of which to recurse on
Cassette.canrecurse(ctx::Type{TypeCtx},::typeof(FCollector)) = false
function Cassette.canrecurse(ctx::TypeCtx,
                             f::Union{typeof(+), typeof(*), typeof(/), typeof(-),typeof(Base.iterate),
                                      typeof(Base.sum),
                                      typeof(Base.mapreduce),
                                      typeof(Base.Broadcast.copy),
                                      typeof(Base.Broadcast.instantiate),
                                      typeof(Base.Math.throw_complex_domainerror),
                                      typeof(Base.Broadcast.broadcasted)},
                             args...)
    return false
end
                
"""    buildgraph

internal function used in the typegraphfrompath
takes the collector object and returns a metagraph
            
"""
function buildgraph(g,collector)
    try
        g[collector.frame.args,:label]
    catch
        add_vertex!(g,:label,collector.frame.args)
    end

    try 
        g[collector.frame.ret,:label]
    catch
        add_vertex!(g,:label,collector.frame.ret)
    end
    
    if !has_edge(g,g[collector.frame.args,:label],g[collector.frame.ret,:label])    
        add_edge!(g,g[collector.frame.args,:label],g[collector.frame.ret,:label],:label,collector.frame.func)
    end
                    
    for frame in collector.data
        buildgraph(g,frame)
    end
                    
    return g
end

"""    cleangraph(g::MetaDiGraph)

removes the nothings to replace them with strings for outputing an image
"""
function cleangraph!(mg::MetaDiGraph)
    for vertex in vertices(mg)
        if get_prop(mg,vertex,:label) == nothing
            set_prop!(mg,vertex,:label,"missing")
        end
    end
    
    for edge in edges(mg)
        if get_prop(mg,edge,:label) == nothing
            set_prop!(mg,edge,:label,"missing")
        end
    end
end
                            

"""    typegraph(path::AbstractString,maxdepth::Int)
            
This is a function that takes in an array of script and produces a MetaDiGraph descibing the system.
takes in optional parameter of recursion depth on the stacktrace defaulted to 3

"""
function typegraph(m::Module,maxdepth::Int=3)
    
    extractor = FCollector(maxdepth, Frame(nothing, (), nothing,)) # init the collector object     
    ctx = TypeCtx(metadata = extractor);     # init the context we want             
    Cassette.overdub(ctx,m.main);    # run the script internally and build the extractor data structure
    g = MetaDiGraph()    # crete a graph where we will init our tree
    set_indexing_prop!(g,:label)    # we want to set this metagraph to be able to index by the names
    g = buildgraph(g,extractor)    # pass the collector ds to make the acutal metagraph
    return cleangraph!(g)
    
end

typegraph

In [5]:
module test

function f(x)
    if x > 3 
        x *= 6
    else
        x += 1
    end
end

function g(y)
    (f(y)/2)*2
end

function main()
    sum([g(i) for i in 1:100])
end
    
end # mod

using Main.test

In [16]:
extractor = FCollector(100,Frame(nothing,(),[]))
ctx = TypeCtx(metadata=extractor)
Cassette.overdub(ctx,Main.test.main)

30273.0

In [17]:
typegraph(Main.test,100)

FCollector{Int64,Frame{Nothing,Tuple{},Any},FCollector}(100, Frame{Nothing,Tuple{},Any}(nothing, (), Any[]), FCollector[FCollector{Int64,Frame{typeof(main),Tuple{},Any},FCollector}(99, Frame{typeof(main),Tuple{},Any}(main, (), Any[Float64]), FCollector[FCollector{Int64,Frame{Colon,Tuple{DataType,DataType},Any},FCollector}(98, Frame{Colon,Tuple{DataType,DataType},Any}(Colon(), (Int64, Int64), Any[UnitRange{Int64}]), FCollector[FCollector{Int64,Frame{typeof(apply_type),Tuple{DataType,DataType},Any},FCollector}(97, Frame{typeof(apply_type),Tuple{DataType,DataType},Any}(apply_type, (UnionAll, DataType), Any[DataType]), FCollector[]), FCollector{Int64,Frame{DataType,Tuple{DataType,DataType},Any},FCollector}(97, Frame{DataType,Tuple{DataType,DataType},Any}(UnitRange{Int64}, (Int64, Int64), Any[UnitRange{Int64}]), FCollector[FCollector{Int64,Frame{typeof(apply_type),Tuple{DataType,DataType},Any},FCollector}(96, Frame{typeof(apply_type),Tuple{DataType,DataType},Any}(apply_type, (UnionAll, Data

In [20]:
extractor.frame

Frame{Nothing,Tuple{},Any}(nothing, (), Any[])

In [28]:
function foo(a,b)
    a + b
end

foo (generic function with 1 method)

In [31]:
@code_lowered foo(1,2)

CodeInfo(
│2 1 ─ %1 = a + b
│  └──      return %1
)

In [32]:
@code_lowered 1 + 2 

CodeInfo(
│53 1 ─ %1 = (Base.add_int)(x, y)
│   └──      return %1
)